In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1'

Mounted at /content/drive
/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1


In [ ]:
# modified: only normalize the input instead of x and y
### RM3 and RM2 version
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.layers import TimeDistributed, Dense, LSTM, RepeatVector, Dropout, Input, Flatten,Concatenate
import numpy as np
import pandas as pd
from tensorflow import keras

import keras
import matplotlib.pyplot as plt
import glob,os
import random
from keras.models import Model
from keras import optimizers
from tensorflow import optimizers
import os,shutil
# read model
from keras.models import load_model
from sklearn.metrics import r2_score 

def listpath(path, list_name):  # 传入存储的list
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            listdir(file_path, list_name)
        else:
            list_name.append(file_path)
        return list_name


import keras.backend as K
def custom_regional_nse(y_true, y_pred):

    numerator = K.square(y_pred - y_true)  # (batch_size, 2)
    dominator = K.square(y_true - K.mean(y_true)) + 0.1
    # summing both loss values along batch dimension 
    numerator_sum = K.sum(numerator, axis=1)        # (batch_size,)
    dominator_sum = K.sum(dominator, axis=1)        # (batch_size,)
    loss = (numerator_sum)/(dominator_sum)
    return loss # automatical average along batch size when optimization


In [ ]:
def split_pub_rm2(static,dataset, n_input, n_output, scaler,shuffle): # 
    x1, x2,y = list(),list(), list()
    for j in range(len(dataset)):
        data_1 = pd.read_csv(dataset[j], index_col = 0)
        data_2 = pd.read_csv(dataset[j], index_col = 0) # not normalize y(target)
        # MinMax for individual event
        sequence = scaler.transform(data_1)
        cat = dataset[j].split('-')[3]
        catchment = cat.split('.')[0]
        temp_static = np.zeros(( n_input + n_output,14))

        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix1 = i + n_input # runoff
            end_ix2 = i + n_input + n_output # others
            end_y = end_ix1 + n_output
            # check if we are beyond the sequence
            if end_ix1 > len(sequence):
                break
            if end_ix2 > len(sequence):
                break
            if end_y > len(sequence):
                break
            # gather input and output parts of the pattern
            temp_x2 = sequence[:,1:][i:end_ix2, :] # 3 rainfall, tmax and tmin -- add static to each time step
            seq_x1, seq_y = sequence[:,0][i:end_ix1], data_2.values[:,0][end_ix1:end_y] # 24 hours runoff and target runoff -- 0 to -1

            # concatnate temp_static and seq_x2
            seq_x2 = np.concatenate((temp_x2,temp_static), axis= 1) # axis= 0, row to row
            x1.append(seq_x1)
            x2.append(seq_x2)
            y.append(seq_y)


    #shuffle
    x1, x2,y = np.array(x1), np.array(x2), np.array(y)
    number_list = np.arange(0,len(x1),1)
    random.shuffle(number_list) 
    x1_new = []
    for index in number_list:
        x1_new.append(x1[index])
    x2_new = []
    for index in number_list:
        x2_new.append(x2[index])
    y_new = []
    for index in number_list:
        y_new.append(y[index])
    if shuffle:
        return np.array(x1_new), np.array(x2_new), np.array(y_new)
    else:
        return x1,x2, y

In [ ]:
def split_pub_rm3(static, dataset, n_input, n_output, scaler,shuffle): # 
    x1, x2,y,seq_static = list(),list(), list(), list()
    for j in range(len(dataset)):
        data_1 = pd.read_csv(dataset[j], index_col = 0)
        data_2 = pd.read_csv(dataset[j], index_col = 0) # not normalize y(target)
        # MinMax for individual event
        sequence = scaler.transform(data_1)
        cat = dataset[j].split('-')[3]
        catchment = cat.split('.')[0]
        
        temp_static = np.zeros((1,14))
        temp_static = static[catchment].values # each basin one static input only one step
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix1 = i + n_input # runoff
            end_ix2 = i + n_input + n_output # others
            end_y = end_ix1 + n_output
            # check if we are beyond the sequence
            if end_ix1 > len(sequence):
                break
            if end_ix2 > len(sequence):
                break
            if end_y > len(sequence):
                break
            # gather input and output parts of the pattern
            seq_x2 = sequence[:,1:][i:end_ix2, :] # 3 rainfall, tmax and tmin -- add static to each time step
            seq_x1, seq_y = sequence[:,0][i:end_ix1], data_2.values[:,0][end_ix1:end_y] # 24 hours runoff and target runoff -- 0 to -1
            # concatnate temp_static and seq_x2
            # seq_x2 = np.concatenate((temp_x2,temp_static), axis= 1) # axis= 0, row to row
            x1.append(seq_x1)
            x2.append(seq_x2)
            y.append(seq_y)
            seq_static.append(list(temp_static))

    #shuffle
    x1, x2,y,seq_static = np.array(x1), np.array(x2), np.array(y), np.array(seq_static)
    number_list = np.arange(0,len(x1),1)
    random.shuffle(number_list) 
    x1_new = []
    for index in number_list:
        x1_new.append(x1[index])
    x2_new = []
    for index in number_list:
        x2_new.append(x2[index])
    y_new = []
    for index in number_list:
        y_new.append(y[index])
    static_new = []
    for index in number_list:
        static_new.append(seq_static[index])
    if shuffle:
        return np.array(x1_new), np.array(x2_new), np.array(y_new), np.array(static_new)
    else:
        return x1,x2, y, static_new

In [ ]:
# leave one out -- for cluster-based pub model
#nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub_all.csv',
 #                       index_col = 0)
cluster_k = 1
nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub_cluster%s.csv' 
                        % cluster_k, index_col = 0) # for cluster-based
basins = nse_table.columns

from sklearn.model_selection import LeaveOneOut # leave-one out kfolder
loo = LeaveOneOut() # random_state to fix the order of shuffle
train_index_list = []
test_index_list = []
# all the train and test will be done under this for loop
for train_index, test_index in loo.split(basins):
    #print('X_train:%s ' % basins[train_index])
    train_index_list.append(train_index)
    test_index_list.append(test_index)


# add static variable to each time step -- pre-normalization not minmax -- negative value here for PCA scores
# read static
pca_score = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/pca_score.csv',index_col = 0)
# normalization -1 - 1 by each PC
temp2 = pca_score.values
temp_nor = preprocessing.normalize(temp2, norm='l2',axis = 0) # default for each column axis= 1
pca_score_nor = pd.DataFrame(temp_nor, index=[pca_score.index], columns=pca_score.columns)

In [ ]:
train_index_list

[array([1, 2, 3, 4]),
 array([0, 2, 3, 4]),
 array([0, 1, 3, 4]),
 array([0, 1, 2, 4]),
 array([0, 1, 2, 3])]

In [ ]:

#for k in range(6,8):
k = 1
num_ite = 1 # count the number of folder
print (num_ite)

train_sub_index = train_index_list[k] # try the first folder
train_file_all = []
for i in range(len(train_sub_index)):
    catchment = basins[train_sub_index[i]]
    train_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
    trainlist = os.listdir(train_path)
    temp_name = []
    for i in range(len(trainlist)):
      temp = train_path + '/'+ trainlist[i]
      train_file_all.append(temp)

test_sub_index = test_index_list[k]
test_file_all = []
for i in range(len(test_sub_index)):
    catchment = basins[test_sub_index[i]]
    test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
    testlist = os.listdir(test_path)
    temp_name = []
    for i in range(len(testlist)):
      temp = test_path + '/'+ testlist[i]
      test_file_all.append(temp)


# dataset split -- train_file_all
# all data should use this scaler--fit the scaler using all data
df2 = pd.DataFrame()
for j in range(len(train_file_all)):
    data = pd.read_csv(train_file_all[j], index_col = 0)
    df2 = pd.concat([df2,data],axis=0) 
temp = df2.values
min_max_scaler1 = MinMaxScaler() # default for each column
min_max_scaler1.fit(temp) # only fit--apply this to the sequence window for normalization
# input for training model

train1_x, train2_x, train_y, train_static = split_pub_rm3(pca_score_nor, train_file_all, 24, 6,min_max_scaler1,shuffle =True)
#train1_x, train2_x, train_y = split_multi_sequence_keras(train_val, 24, 6,min_max_scaler1,shuffle =True)

# input for validation model -- using test
val1_x, val2_x, val_y, val_static = split_pub_rm3(pca_score_nor, test_file_all, 24, 6, min_max_scaler1,shuffle =True)

1


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted wi

In [ ]:
train_y.shape

(12508, 6)

In [ ]:
######################## first folder ##########################
###  ###

# Based on RM3 setting and for cluster-based model -- static variable from seperate layer

for k in range(len(train_index_list)):
#for k in range(6,8):
    num_ite = k + 1 # count the number of folder
    print (num_ite)

    train_sub_index = train_index_list[k] # try the first folder
    train_file_all = []
    for i in range(len(train_sub_index)):
        catchment = basins[train_sub_index[i]]
        train_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        trainlist = os.listdir(train_path)
        temp_name = []
        for i in range(len(trainlist)):
          temp = train_path + '/'+ trainlist[i]
          train_file_all.append(temp)

    test_sub_index = test_index_list[k]
    test_file_all = []
    for i in range(len(test_sub_index)):
        catchment = basins[test_sub_index[i]]
        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)
        temp_name = []
        for i in range(len(testlist)):
          temp = test_path + '/'+ testlist[i]
          test_file_all.append(temp)


    # dataset split -- train_file_all
    # all data should use this scaler--fit the scaler using all data
    df2 = pd.DataFrame()
    for j in range(len(train_file_all)):
        data = pd.read_csv(train_file_all[j], index_col = 0)
        df2 = pd.concat([df2,data],axis=0) 
    temp = df2.values
    min_max_scaler1 = MinMaxScaler() # default for each column
    min_max_scaler1.fit(temp) # only fit--apply this to the sequence window for normalization
    # input for training model

    train1_x, train2_x, train_y, train_static = split_pub_rm3(pca_score_nor, train_file_all, 24, 6,min_max_scaler1,shuffle =True)
    #train1_x, train2_x, train_y = split_multi_sequence_keras(train_val, 24, 6,min_max_scaler1,shuffle =True)

    # input for validation model -- using test
    val1_x, val2_x, val_y, val_static = split_pub_rm3(pca_score_nor, test_file_all, 24, 6, min_max_scaler1,shuffle =True)
    #val1_x, val2_x, val_y = split_multi_sequence_keras(test, 24, 6, min_max_scaler1,shuffle =True)

    val1_x = val1_x.reshape((val1_x.shape[0],val1_x.shape[1],1)) 
    train1_x = train1_x.reshape((train1_x.shape[0],train1_x.shape[1],1))
    ### only necessary when encoder layer is lstm
    #train_static = train_static.reshape((train_static.shape[0],1,train_static.shape[1]))
    #val_static = val_static.reshape((val_static.shape[0],1,val_static.shape[1]))

    # define customer optimizer
    #RMSprop=keras.optimizers.RMSprop(lr=0.0001)
    RMSprop= optimizers.RMSprop(lr=0.0001)

    # define special callbacks
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    ######## early longer ########
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=2)
    # early stop 50
    # design network
    dim_dense=[512, 256, 256, 128, 64]

    # input of runoff observation, LSTM encoder
    input_1 = Input(shape=(train1_x.shape[1],train1_x.shape[2]), name='LSTM1_input') # shape should be 72*1 for runoff observation
    LSTM1 = LSTM(256, return_sequences=False)(input_1)

    # input of runoff observation and forecast, LSTM encoder
    input_2 = Input(shape=(train2_x.shape[1],train2_x.shape[2]), name='LSTM2_input') # shape should be (72+24)*n=96*n, for rainfall observation (72) and predictions (24) for n stations (if there is no upstream station, n=1)
    LSTM2 = LSTM(256, return_sequences=False)(input_2)

    # input of other non-timeseries data, such as daily or monthly data.
    input_static = Input(shape=(14,), name='static_input') # shape = 14 -- 14 static variables -- 2D in this case with only Input or Dense layer (14,)/3D for LSTM layer (1,14)
    Layer3 = Dense(16, activation='relu')(input_static)
    #LSTM_static = LSTM(256, return_sequences=False)(input_static) # using LSTM for feeding static variable


    # connect all data
    x = Concatenate()([Layer3, LSTM1, LSTM2]) # Get state vector.default is axis = -1
    x = RepeatVector(6)(x)  # 6 hour ahead

    # LSTM decoder
    x = LSTM(512, return_sequences=True)(x)

    # final fully-connected layer for final result
    for dim in dim_dense:
      x = TimeDistributed(Dense(dim, activation='relu'))(x)
      x = TimeDistributed(Dropout(0.4))(x) # Some dropout for dense layers.
    main_out = TimeDistributed(Dense(1, activation='relu'))(x) 
    main_out = Flatten()(main_out)
    #################################### nse mse ####################
    model = Model(inputs=[input_static, input_1, input_2], outputs=main_out)
    #model.compile(optimizer=RMSprop, loss='mse')
    model.compile(optimizer=RMSprop, loss=custom_regional_nse) 
    model.summary()

    # fit network -- when RM3 -- add train and val static variables directly
    history = model.fit([train_static, train1_x, train2_x], train_y, epochs=300, batch_size=128,
              validation_data=([val_static, val1_x, val2_x], val_y), callbacks=[reduce_lr,early_stopping], verbose=1)

    test_sub_index = test_index_list[k]
    catchment = basins[test_sub_index]
    from keras.models import load_model
    model.save('/content/drive/My Drive/master_thesis/_code/regional_model/pub_rm3_cluster%s_%s.h5' % (cluster_k,catchment)) 
    # last number indicate the num of folder

    ######### modification ##########

    model_pub = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/pub_rm3_cluster%s_%s.h5' % (cluster_k, catchment), 
                           custom_objects={'custom_regional_nse': custom_regional_nse})
    #model_pub = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/pub_south_%s.h5' % num_ite)
    #nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub1.csv',index_col = 0)
    basins = nse_table.columns
    
    test_file_all = []
    for i in range(len(test_sub_index)): # test on test basins

        catchment = basins[test_sub_index[i]]

        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)

        dataset = testlist
        ed_nse = []
        ed_hour = []
        ed_volume = []
        ed_peak = []
        dates = []

        for j in range(len(dataset)):
            x1, x2,y, seq_static = list(),list(), list(), list()
            data_1 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0)
            data_2 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0) # not normalize y(target)
            date = dataset[j][:10]
            # MinMax for individual event
            n_input = 24
            n_output = 6
            sequence = min_max_scaler1.transform(data_1)

            # add static variable
            temp_static = np.zeros((1,14))
            #temp_static = np.zeros((1,27))
            temp_static = pca_score_nor[catchment].values # each basin one static input only one step
            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix1 = i*n_output + n_input # runoff
                end_ix2 = i*n_output + n_input + n_output # others
                end_y = end_ix1 + n_output
                # check if we are beyond the sequence
                if end_ix1 > len(sequence):
                    break
                if end_ix2 > len(sequence):
                    break
                if end_y > len(sequence):
                    break
                # gather input and output parts of the pattern
                seq_x2 = sequence[:,1:][i*n_output:end_ix2, :] # 3 rainfall, tmax and tmin -- add static to each time step
                seq_x1, seq_y = sequence[:,0][i*n_output:end_ix1], data_2.values[:,0][end_ix1:end_y] # 24 hours runoff and target runoff -- 0 to -1
                # concatnate temp_static and seq_x2

                #seq_x2 = np.concatenate((temp_x2,temp_static), axis= 1) # axis= 0, row to row
                x1.append(seq_x1)
                x2.append(seq_x2)
                y.append(seq_y)
                seq_static.append(list(temp_static))

            #x1, x2,y = np.array(x1), np.array(x2), np.array(y)
            x1, x2,y,seq_static = np.array(x1), np.array(x2), np.array(y), np.array(seq_static)
            predictions = []
            x1 = x1.reshape((x1.shape[0],x1.shape[1],1))
            #seq_static = seq_static.reshape((seq_static.shape[0],1,seq_static.shape[1]))
            predict = model_pub.predict([seq_static, x1,x2]) # need be replaced
            predictions.append(predict)
            dates.append(date) #
            ed_peak.append(np.max(np.array(predictions).flatten()))
            ed_nse.append(r2_score(np.array(y).flatten(), np.array(predictions).flatten()))

            # peak hour
            hour_pre = np.argmax(np.array(predictions).flatten())
            hour_obs = np.argmax(np.array(y).flatten())
            ed_hour.append((hour_pre - hour_obs))
            
            # peak discharge
            max_pre = np.max(np.array(predictions).flatten())
            max_obs = np.max(np.array(y).flatten())
            ed_volume.append((max_pre - max_obs)/max_obs)

        temp = [dates, ed_peak, ed_hour, ed_volume] ###
        temp = np.array(temp)
        output = pd.DataFrame([temp[0,:],temp[1,:],temp[2,:],temp[3,:]], index=['date', 'flow', 'hour','ratio'])
        output.to_csv('/content/drive/My Drive/master_thesis/_code/regional_model/peak_rm3_cluster%s/%s_peak_pub.csv' % (cluster_k,catchment))
        nse_table[catchment] = np.median(ed_nse)

In [ ]:
temp

In [ ]:
nse_table.to_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/nse_rm3_cluster%s.csv' % cluster_k)

In [ ]:
# PCA value
# modified: only normalize the input instead of x and y
def split_pub_rm2(static, dataset, n_input, n_output, scaler,shuffle): # add static table and select the repective variables
    x1, x2,y = list(),list(), list()
    for j in range(len(dataset)):
        data_1 = pd.read_csv(dataset[j], index_col = 0)
        data_2 = pd.read_csv(dataset[j], index_col = 0) # not normalize y(target)
        #data_1 = pd.read_csv(path + dataset[j], index_col = 0)
        #data_2 = pd.read_csv(path + dataset[j], index_col = 0) # not normalize y(target)
        # MinMax for individual event
        sequence = scaler.transform(data_1)
        ####################27 catch##################
        temp_static = np.zeros(( n_input + n_output,14))
        #temp_static = np.zeros(( n_input + n_output,27))
        cat = dataset[j].split('-')[3]
        catchment = cat.split('.')[0]
        for i in range(temp_static.shape[1]):
            temp_static[i,:] = static[catchment].values
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix1 = i + n_input # runoff
            end_ix2 = i + n_input + n_output # others
            end_y = end_ix1 + n_output
            # check if we are beyond the sequence
            if end_ix1 > len(sequence):
                break
            if end_ix2 > len(sequence):
                break
            if end_y > len(sequence):
                break
            # gather input and output parts of the pattern
            temp_x2 = sequence[:,1:][i:end_ix2, :] # 3 rainfall, tmax and tmin -- add static to each time step
            seq_x1, seq_y = sequence[:,0][i:end_ix1], data_2.values[:,0][end_ix1:end_y] # 24 hours runoff and target runoff -- 0 to -1
            # concatnate temp_static and seq_x2

            seq_x2 = np.concatenate((temp_x2,temp_static), axis= 1) # axis= 0, row to row
            x1.append(seq_x1)
            x2.append(seq_x2)
            y.append(seq_y)

#shuffle
    x1, x2,y = np.array(x1), np.array(x2), np.array(y)
    number_list = np.arange(0,len(x1),1)
    random.shuffle(number_list) 
    x1_new = []
    for index in number_list:
        x1_new.append(x1[index])
    x2_new = []
    for index in number_list:
        x2_new.append(x2[index])
    y_new = []
    for index in number_list:
        y_new.append(y[index])
    if shuffle:
        return np.array(x1_new), np.array(x2_new), np.array(y_new)
    else:
        return x1,x2, y

In [ ]:
######################## first folder ################################
cluster_k = 1 # choose the cluster
# Based on # RM2 setting -- PUB model with static variable but directly concatanate into sequence
for k in range(len(train_index_list)):
#for k in range(6,8):
    num_ite = k + 1 # count the number of folder
    print (num_ite)

    train_sub_index = train_index_list[k] # try the first folder
    train_file_all = []
    for i in range(len(train_sub_index)):
        catchment = basins[train_sub_index[i]]
        train_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        trainlist = os.listdir(train_path)
        temp_name = []
        for i in range(len(trainlist)):
          temp = train_path + '/'+ trainlist[i]
          train_file_all.append(temp)

    test_sub_index = test_index_list[k]
    test_file_all = []
    for i in range(len(test_sub_index)):
        catchment = basins[test_sub_index[i]]
        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)
        temp_name = []
        for i in range(len(testlist)):
          temp = test_path + '/'+ testlist[i]
          test_file_all.append(temp)

    # dataset split -- train_file_all
    # all data should use this scaler--fit the scaler using all data
    df2 = pd.DataFrame()
    for j in range(len(train_file_all)):
        data = pd.read_csv(train_file_all[j], index_col = 0)
        df2 = pd.concat([df2,data],axis=0) 

    temp = df2.values
    min_max_scaler1 = MinMaxScaler() # default for each column
    min_max_scaler1.fit(temp) # only fit--apply this to the sequence window for normalization
    # input for training model

    train1_x, train2_x, train_y = split_pub_rm2(pca_score_nor, train_file_all, 24, 6,min_max_scaler1,shuffle =True)
    #train1_x, train2_x, train_y = split_multi_sequence_keras(train_val, 24, 6,min_max_scaler1,shuffle =True)

    # input for validation model -- using test
    val1_x, val2_x, val_y = split_pub_rm2(pca_score_nor, test_file_all, 24, 6, min_max_scaler1,shuffle =True)
    #val1_x, val2_x, val_y = split_multi_sequence_keras(test, 24, 6, min_max_scaler1,shuffle =True)

    val1_x = val1_x.reshape((val1_x.shape[0],val1_x.shape[1],1)) 
    train1_x = train1_x.reshape((train1_x.shape[0],train1_x.shape[1],1))
    ### only necessary when encoder layer is lstm
    #train_static = train_static.reshape((train_static.shape[0],1,train_static.shape[1]))
    #val_static = val_static.reshape((val_static.shape[0],1,val_static.shape[1]))

    # define customer optimizer
    #RMSprop=keras.optimizers.RMSprop(lr=0.0001)
    RMSprop=optimizers.RMSprop(lr=0.0001)

    # define special callbacks
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    ######## early longer ########
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=2)
    # early stop 50
    # design network
    dim_dense=[512, 256, 256, 128, 64]

    # input of runoff observation, LSTM encoder
    input_1 = Input(shape=(train1_x.shape[1],train1_x.shape[2]), name='LSTM1_input') # shape should be 72*1 for runoff observation
    LSTM1 = LSTM(256, return_sequences=False)(input_1)

    # input of runoff observation and forecast, LSTM encoder
    input_2 = Input(shape=(train2_x.shape[1],train2_x.shape[2]), name='LSTM2_input') # shape should be (72+24)*n=96*n, for rainfall observation (72) and predictions (24) for n stations (if there is no upstream station, n=1)
    LSTM2 = LSTM(256, return_sequences=False)(input_2)

    # input of other non-timeseries data, such as daily or monthly data.
    #input_static = Input(shape=(14,), name='static_input') # shape = 14 -- 14 static variables -- 2D in this case with only Input or Dense layer (14,)/3D for LSTM layer (1,14)
    #Layer3 = Dense(16, activation='relu')(input_static)
    #LSTM_static = LSTM(256, return_sequences=False)(input_static) # using LSTM for feeding static variable

    # connect all data
    x = Concatenate()([LSTM1, LSTM2]) # Get state vector.default is axis = -1
    x = RepeatVector(6)(x)  # 6 hour ahead

    # LSTM decoder
    x = LSTM(512, return_sequences=True)(x)

    # final fully-connected layer for final result
    for dim in dim_dense:
      x = TimeDistributed(Dense(dim, activation='relu'))(x)
      x = TimeDistributed(Dropout(0.4))(x) # Some dropout for dense layers.
    main_out = TimeDistributed(Dense(1, activation='relu'))(x) 
    main_out = Flatten()(main_out)
    #################################### nse mse ####################
    model = Model(inputs=[input_1, input_2], outputs=main_out)
    #model.compile(optimizer=RMSprop, loss='mse')
    model.compile(optimizer=RMSprop, loss=custom_regional_nse) 
    model.summary()

    # fit network -- when RM3 -- add train and val static variables directly
    history = model.fit([train1_x, train2_x], train_y, epochs=300, batch_size=128,
              validation_data=([val1_x, val2_x], val_y), callbacks=[reduce_lr,early_stopping], verbose=1)

    test_sub_index = test_index_list[k]
    catchment = basins[test_sub_index]
    from keras.models import load_model
    model.save('/content/drive/My Drive/master_thesis/_code/regional_model/pub_rm2_cluster%s_%s.h5' % (cluster_k,catchment)) 
    # last number indicate the num of folder

    ######### modification ##########

    model_pub = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/pub_rm2_cluster%s_%s.h5' % (cluster_k, catchment), 
                           custom_objects={'custom_regional_nse': custom_regional_nse})
    #model_pub = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/pub_south_%s.h5' % num_ite)
    #nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub1.csv',index_col = 0)
    basins = nse_table.columns
    
    test_file_all = []
    for i in range(len(test_sub_index)): # test on test basins

        catchment = basins[test_sub_index[i]]

        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)

        dataset = testlist
        ed_nse = []
        ed_hour = []
        ed_volume = []
        ed_peak = []
        dates = []

        for j in range(len(dataset)):
            x1, x2,y, seq_static = list(),list(), list(), list()
            data_1 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0)
            data_2 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0) # not normalize y(target)
            date = dataset[j][:10]
            # MinMax for individual event
            n_input = 24
            n_output = 6
            sequence = min_max_scaler1.transform(data_1)

            # add static variable
            temp_static = np.zeros(( n_input + n_output,14))
            #temp_static = np.zeros((1,27))
            for i in range(temp_static.shape[1]):
              temp_static[i,:] = pca_score_nor[catchment].values
            #temp_static = pca_score_nor[catchment].values # each basin one static input only one step
            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix1 = i*n_output + n_input # runoff
                end_ix2 = i*n_output + n_input + n_output # others
                end_y = end_ix1 + n_output
                # check if we are beyond the sequence
                if end_ix1 > len(sequence):
                    break
                if end_ix2 > len(sequence):
                    break
                if end_y > len(sequence):
                    break
                # gather input and output parts of the pattern
 
                temp_x2 = sequence[:,1:][i*n_output:end_ix2, :] # 3 rainfall, tmax and tmin -- add static to each time step
                seq_x1, seq_y = sequence[:,0][i*n_output:end_ix1], data_2.values[:,0][end_ix1:end_y] # 24 hours runoff and target runoff -- 0 to -1
                # concatnate temp_static and seq_x2
                seq_x2 = np.concatenate((temp_x2,temp_static), axis= 1) # axis= 0, row to row
                #seq_x2 = np.concatenate((temp_x2,temp_static), axis= 1) # axis= 0, row to row
                x1.append(seq_x1)
                x2.append(seq_x2)
                y.append(seq_y)

            #x1, x2,y = np.array(x1), np.array(x2), np.array(y)
            x1, x2, y = np.array(x1), np.array(x2), np.array(y)
            predictions = []
            x1 = x1.reshape((x1.shape[0],x1.shape[1],1))
            #seq_static = seq_static.reshape((seq_static.shape[0],1,seq_static.shape[1]))
            predict = model_pub.predict([x1,x2]) # need be replaced
            predictions.append(predict)
            dates.append(date) #
            ed_peak.append(np.max(np.array(predictions).flatten()))
            ed_nse.append(r2_score(np.array(y).flatten(), np.array(predictions).flatten()))

            # peak hour
            hour_pre = np.argmax(np.array(predictions).flatten())
            hour_obs = np.argmax(np.array(y).flatten())
            ed_hour.append((hour_pre - hour_obs))
            
            # peak discharge
            max_pre = np.max(np.array(predictions).flatten())
            max_obs = np.max(np.array(y).flatten())
            ed_volume.append((max_pre - max_obs)/max_obs)

        temp = [dates, ed_peak, ed_hour, ed_volume] ###
        temp = np.array(temp)
        output = pd.DataFrame([temp[0,:],temp[1,:],temp[2,:],temp[3,:]], index=['date', 'flow', 'hour','ratio'])
        output.to_csv('/content/drive/My Drive/master_thesis/_code/regional_model/peak_rm2_cluster%s/%s_peak_pub.csv' % (cluster_k,catchment))
        nse_table[catchment] = np.median(ed_nse)

1


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted wi

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 LSTM1_input (InputLayer)       [(None, 24, 1)]      0           []                               
                                                                                                  
 LSTM2_input (InputLayer)       [(None, 30, 17)]     0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 256)          264192      ['LSTM1_input[0][0]']            
                                                                                                  
 lstm_1 (LSTM)                  (None, 256)          280576      ['LSTM2_input[0][0]']            
                                                                                              

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted wi

2


...

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted wi

5


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted wi

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 LSTM1_input (InputLayer)       [(None, 24, 1)]      0           []                               
                                                                                                  
 LSTM2_input (InputLayer)       [(None, 30, 17)]     0           []                               
                                                                                                  
 lstm_12 (LSTM)                 (None, 256)          264192      ['LSTM1_input[0][0]']            
                                                                                                  
 lstm_13 (LSTM)                 (None, 256)          280576      ['LSTM2_input[0][0]']            
                                                                                            

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted wi

In [ ]:
nse_table.to_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/nse_rm2_cluster%s.csv' % cluster_k)

In [ ]:
# for all catchments PUB
nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub_all.csv',index_col = 0)
basins = nse_table.columns

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle = True,random_state = 1) # random_state to fix the order of shuffle
train_index_list = []
test_index_list = []
# all the train and test will be done under this for loop
for train_index, test_index in kf.split(basins):
    #print('X_train:%s ' % basins[train_index])
    train_index_list.append(train_index)
    test_index_list.append(test_index)


In [ ]:
test_index_list

In [ ]:
######################## first folder ##########################
###  ###
# leave for other use -- case for all the catchment

for k in range(len(train_index_list)):
#for k in range(6,8):
    num_ite = k + 1 # count the number of folder
    print (num_ite)

    train_sub_index = train_index_list[k] # try the first folder
    train_file_all = []
    for i in range(len(train_sub_index)):
        catchment = basins[train_sub_index[i]]
        train_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        trainlist = os.listdir(train_path)
        temp_name = []
        for i in range(len(trainlist)):
          temp = train_path + '/'+ trainlist[i]
          train_file_all.append(temp)

    test_sub_index = test_index_list[k]
    test_file_all = []
    for i in range(len(test_sub_index)):
        catchment = basins[test_sub_index[i]]
        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)
        temp_name = []
        for i in range(len(testlist)):
          temp = test_path + '/'+ testlist[i]
          test_file_all.append(temp)


    # dataset split -- train_file_all
    # all data should use this scaler--fit the scaler using all data
    df2 = pd.DataFrame()
    for j in range(len(train_file_all)):
        data = pd.read_csv(train_file_all[j], index_col = 0)
        df2 = pd.concat([df2,data],axis=0) 
    temp = df2.values
    min_max_scaler1 = MinMaxScaler() # default for each column
    min_max_scaler1.fit(temp) # only fit--apply this to the sequence window for normalization
    # input for training model
    train1_x, train2_x, train_y = split_pub(train_file_all, 24, 6,min_max_scaler1,shuffle =True)
    #train1_x, train2_x, train_y = split_multi_sequence_keras(train_val, 24, 6,min_max_scaler1,shuffle =True)

    # input for validation model -- using test
    val1_x, val2_x, val_y = split_pub(test_file_all, 24, 6, min_max_scaler1,shuffle =True)
    #val1_x, val2_x, val_y = split_multi_sequence_keras(test, 24, 6, min_max_scaler1,shuffle =True)

    val1_x = val1_x.reshape((val1_x.shape[0],val1_x.shape[1],1)) 
    train1_x = train1_x.reshape((train1_x.shape[0],train1_x.shape[1],1))
    ### only necessary when encoder layer is lstm
    #train_static = train_static.reshape((train_static.shape[0],1,train_static.shape[1]))
    #val_static = val_static.reshape((val_static.shape[0],1,val_static.shape[1]))

    # define customer optimizer
    RMSprop=keras.optimizers.RMSprop(lr=0.0001)

    # define special callbacks
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    ######## early longer ########
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=2)
    # early stop 50
    # design network
    dim_dense=[512, 256, 256, 128, 64]

    # input of runoff observation, LSTM encoder
    input_1 = Input(shape=(train1_x.shape[1],train1_x.shape[2]), name='LSTM1_input') # shape should be 72*1 for runoff observation
    LSTM1 = LSTM(256, return_sequences=False)(input_1)

    # input of runoff observation and forecast, LSTM encoder
    input_2 = Input(shape=(train2_x.shape[1],train2_x.shape[2]), name='LSTM2_input') # shape should be (72+24)*n=96*n, for rainfall observation (72) and predictions (24) for n stations (if there is no upstream station, n=1)
    LSTM2 = LSTM(256, return_sequences=False)(input_2)

    # input of other non-timeseries data, such as daily or monthly data.
    #input_static = Input(shape=(14,), name='static_input') # shape = 14 -- 14 static variables -- 2D in this case with only Input or Dense layer (14,)/3D for LSTM layer (1,14)
    #Layer3 = Dense(16, activation='relu')(input_static)
    #LSTM_static = LSTM(256, return_sequences=False)(input_static)


    # connect all data
    x = Concatenate()([LSTM1, LSTM2]) # Get state vector.default is axis = -1
    x = RepeatVector(6)(x)  # 6 hour ahead

    # LSTM decoder
    x = LSTM(512, return_sequences=True)(x)

    # final fully-connected layer for final result
    for dim in dim_dense:
      x = TimeDistributed(Dense(dim, activation='relu'))(x)
      x = TimeDistributed(Dropout(0.4))(x) # Some dropout for dense layers.
    main_out = TimeDistributed(Dense(1, activation='relu'))(x) 
    main_out = Flatten()(main_out)
    #################################### nse mse ####################
    model = Model(inputs=[input_1, input_2], outputs=main_out)
    #model.compile(optimizer=RMSprop, loss='mse')
    model.compile(optimizer=RMSprop, loss=custom_regional_nse) 
    model.summary()

    # fit network
    history = model.fit([train1_x, train2_x], train_y, epochs=300, batch_size=128,
              validation_data=([val1_x, val2_x], val_y), callbacks=[reduce_lr,early_stopping], verbose=1)

    from keras.models import load_model
    model.save('/content/drive/My Drive/master_thesis/_code/regional_model/pub_rm3_%s.h5' % num_ite) # last number indicate the num of folder

    ######### modification ##########
    model_pub = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/pub_rm3_%s.h5' % num_ite, custom_objects={'custom_regional_nse': custom_regional_nse})
    #model_pub = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/pub_south_%s.h5' % num_ite)
    #nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub1.csv',index_col = 0)
    basins = nse_table.columns
    
    test_sub_index = test_index_list[k]
    test_file_all = []

    for i in range(len(test_sub_index)): # test on test basins

        catchment = basins[test_sub_index[i]]

        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)

        dataset = testlist
        ed_nse = []
        ed_hour = []
        ed_volume = []
        ed_peak = []
        dates = []

        for j in range(len(dataset)):
            x1, x2,y, seq_static = list(),list(), list(), list()
            data_1 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0)
            data_2 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0) # not normalize y(target)
            date = dataset[j][:10]
            # MinMax for individual event
            n_input = 24
            n_output = 6
            sequence = min_max_scaler1.transform(data_1)

            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix1 = i*n_output + n_input # runoff
                end_ix2 = i*n_output + n_input + n_output # others
                end_y = end_ix1 + n_output
                # check if we are beyond the sequence
                if end_ix1 > len(sequence):
                    break
                if end_ix2 > len(sequence):
                    break
                if end_y > len(sequence):
                    break
                # gather input and output parts of the pattern
                seq_x2 = sequence[:,1:][i*n_output:end_ix2, :] # 3 rainfall, tmax and tmin -- add static to each time step
                seq_x1, seq_y = sequence[:,0][i*n_output:end_ix1], data_2.values[:,0][end_ix1:end_y] # 24 hours runoff and target runoff -- 0 to -1
                # concatnate temp_static and seq_x2

                #seq_x2 = np.concatenate((temp_x2,temp_static), axis= 1) # axis= 0, row to row
                x1.append(seq_x1)
                x2.append(seq_x2)
                y.append(seq_y)

            x1, x2,y = np.array(x1), np.array(x2), np.array(y)
            predictions = []
            x1 = x1.reshape((x1.shape[0],x1.shape[1],1))
            #seq_static = seq_static.reshape((seq_static.shape[0],1,seq_static.shape[1]))
            predict = model_pub.predict([x1,x2]) # need be replaced
            predictions.append(predict)
            dates.append(date) #
            ed_peak.append(np.max(np.array(predictions).flatten()))
            ed_nse.append(r2_score(np.array(y).flatten(), np.array(predictions).flatten()))

            # peak hour
            hour_pre = np.argmax(np.array(predictions).flatten())
            hour_obs = np.argmax(np.array(y).flatten())
            ed_hour.append((hour_pre - hour_obs))
            
            # peak discharge
            max_pre = np.max(np.array(predictions).flatten())
            max_obs = np.max(np.array(y).flatten())
            ed_volume.append((max_pre - max_obs)/max_obs)

        #temp = [dates, obs_peak, ed_peak, ed_hour, ed_volume] ###
        #temp = np.array(temp)
        #output = pd.DataFrame([temp[0,:],temp[1,:],temp[2,:],temp[3,:]], index=['date', 'flow', 'hour','ratio'])
        #output.to_csv('/content/drive/My Drive/master_thesis/_code/regional_model/peak_pub2/%s_peak_pub3.csv' % catchment)
        nse_table[catchment] = np.median(ed_nse)

In [ ]:
nse_table.to_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/nse_pub_rm1_2.csv')

In [ ]:
nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub_all.csv',index_col = 0)
basins = nse_table.columns

from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle = True,random_state = 1) # random_state to fix the order of shuffle
train_index_list = []
test_index_list = []
# all the train and test will be done under this for loop
for train_index, test_index in kf.split(basins):
#for test_index, train_index in kf.split(basins): # reverse so we have 12 for train 24 for test
    #print('X_train:%s ' % basins[train_index])
    train_index_list.append(train_index)
    test_index_list.append(test_index)


# add static variable to each time step -- pre-normalization not minmax -- negative value here for PCA scores
# read static
pca_score = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/pca_score.csv',index_col = 0)
# normalization -1 - 1 by each PC
temp2 = pca_score.values
temp_nor = preprocessing.normalize(temp2, norm='l2',axis = 1) # default for each column axis= 1
pca_score_nor = pd.DataFrame(temp_nor, index=[pca_score.index], columns=pca_score.columns)

In [ ]:
output = pd.DataFrame(temp.T, columns=['date','obs','flow', 'hour','ratio'])

In [ ]:
# leave one out -- for cluster-based pub model
#nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub_all.csv',
 #                       index_col = 0)
cluster = 1
nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name_pub_cluster%s.csv' 
                        % cluster, index_col = 0) # for cluster-based
basins = nse_table.columns

from sklearn.model_selection import LeaveOneOut # leave-one out kfolder
loo = LeaveOneOut() # random_state to fix the order of shuffle
train_index_list = []
test_index_list = []
# all the train and test will be done under this for loop
for train_index, test_index in loo.split(basins):
    #print('X_train:%s ' % basins[train_index])
    train_index_list.append(train_index)
    test_index_list.append(test_index)


# add static variable to each time step -- pre-normalization not minmax -- negative value here for PCA scores
# read static
pca_score = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/pca_score.csv',index_col = 0)
# normalization -1 - 1 by each PC
temp2 = pca_score.values
temp_nor = preprocessing.normalize(temp2, norm='l2',axis = 0) # default for each column axis= 1
pca_score_nor = pd.DataFrame(temp_nor, index=[pca_score.index], columns=pca_score.columns)

In [ ]:
pca_score_nor